In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import gc

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch.optim as optim
import re

from torchtext.prototype.generate import GenerationUtils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
#load data from link
data = pd.read_csv('https://raw.githubusercontent.com/Pulsy1577/Deep-Learning-Project/main/hogwarts_legacy_reviews.csv')

In [ ]:
class CustomTextDataset():
  def __init__(self, text, labels):
      self.labels = labels
      self.text = text

  def __len__(self):
          return len(self.labels)

  def __getitem__(self, idx):
          label = self.labels[idx]
          text = self.text[idx]
          sample = {"Text": text, "Class": label}
          return sample


In [ ]:
# define data and class labels
numpyData = data.to_numpy()

text_tot = numpyData[:,3]
labels_tot = numpyData[:,2]

#splitting train and test set
train_text = text_tot[0:34802]
test_text = text_tot[34802:]

train_labels = labels_tot[0:34802]
test_labels = labels_tot[34802:]

# create Pandas DataFrame
text_labels_df_train = pd.DataFrame({'Text': train_text, 'Labels': train_labels})
text_labels_df_test = pd.DataFrame({'Text': test_text, 'Labels': test_labels})

#apply prefix for T-5
task = "sst2 sentence"
for i in range(len(test_labels)):
  text_labels_df_test["Text"][i] = task + ": " +  text_labels_df_test["Text"][i]

# #convert labels to numbers, 1 is negative, 2 is positive
# for i in range(len(test_labels)):
#   if(text_labels_df_test["Labels"][i] == "Positive"):
#     text_labels_df_test["Labels"][i] = "2"
#   else:
#     text_labels_df_test["Labels"][i] = "1"

# define data set object
TestSet = CustomTextDataset(text_labels_df_test['Text'],text_labels_df_test['Labels'])

def collate_batch(batch):
    word_tensor = torch.tensor([[1.], [0.], [45.]])
    label_tensor = torch.tensor([[1.]])
    
    text_list, classes = [], []
    for (_text, _class) in batch:
        text_list.append(word_tensor)
        classes.append(label_tensor)

    text = torch.cat(text_list)
    classes = torch.tensor(classes)
    return text, classes

#DL_DS = DataLoader(TestSet, batch_size=2, collate_fn=collate_batch)

In [ ]:
# Display text and label.
print('\nFirst iteration of data set: ', next(iter(TestSet)), '\n')
# Print how many items are in the data set
print('Length of data set: ', len(TestSet), '\n')
# Print entire data set
print('Entire data set: ', list(DataLoader(TestSet)), '\n')



First iteration of data set:  {'Text': 'sst2 sentence: just WOW! this game is insanely awesome.. thank you avalanche and thank you everyone :)', 'Class': 'Positive'} 

Length of data set:  11603 

Entire data set:  [{'Text': ['sst2 sentence: just WOW! this game is insanely awesome.. thank you avalanche and thank you everyone :)'], 'Class': ['Positive']}, {'Text': ['sst2 sentence: [u]*Revelio*[/u][spoiler]10/10[/spoiler]'], 'Class': ['Positive']}, {'Text': ['sst2 sentence: AVADA KEDAVRA'], 'Class': ['Positive']}, {'Text': ['sst2 sentence: Dream come true'], 'Class': ['Positive']}, {'Text': ['sst2 sentence: As a Harry Potter fan, this game went above and beyond.  The exploration alone will keep me coming back to this game long after its finished'], 'Class': ['Positive']}, {'Text': ['sst2 sentence: This game is amazing'], 'Class': ['Positive']}, {'Text': ["sst2 sentence: 1/10 Doesn't let me go Total Muggle DeathAt least I can genocide goblins."], 'Class': ['Positive']}, {'Text': ['sst2 s

In [ ]:
#import pre-trained model
from torchtext.models import T5_BASE_GENERATION
#transform = T5_BASE_GENERATION.transform()

#set up model

t5_base = T5_BASE_GENERATION
transform = t5_base.transform()
model = t5_base.get_model()
sequence_generator = GenerationUtils(model)
model.eval()

print()

In [ ]:
# bs = 8
# batch = next(iter(DataLoader(TestSet, batch_size=bs)))
# input_text = batch["Text"]
# target = batch["Class"]
# beam_size = 1

# model.to(device)

# model_input = transform(input_text)
# model_output = sequence_generator.generate(model_input, eos_idx=1, num_beams=beam_size)
# output_text = transform.decode(model_output.tolist())


# for i in range(bs):
#     print(f"Example {i+1}:\n")
#     print(f"input_text: {input_text[i]}\n")
#     print(f"prediction: {output_text[i]}\n")
#     print(f"target: {target[i]}\n\n")


RuntimeError: ignored

In [ ]:



# def predBatch(dataSet, bs):
#   model.to(device)
#   preds = []
#   real = []
#   k = 0

#   while(k <= len(dataSet)):
    
#     if (20 > (len(dataSet) - k)):
#       bs = len(dataSet) - k

#     text_subset = test_text[k:k+bs]
#     label_subset = test_labels[k:k+bs]
    

#     temp = pd.DataFrame({'Text': text_subset, 'Labels': label_subset})
    
#     task = "sst2 sentence"
#     for i in range(len(text_subset)):
#       temp["Text"][i] = task + ": " +  temp["Text"][i]
    
#     subset = CustomTextDataset(temp['Text'],temp['Labels'])

#     batch = next(iter(DataLoader(subset, batch_size=bs)))
#     input_text = batch["Text"]
#     target = batch["Class"]
#     beam_size = 1

#     model_input = transform(input_text)
#     model_output = sequence_generator.generate(model_input, eos_idx=1, num_beams=beam_size)
#     output_text = transform.decode(model_output.tolist())
#     k += bs

#     print(k)
#     for i in range(bs):
#       # print(f"prediction: {output_text[i]}\n")
#       # print(f"target: {target[i]}\n\n")
#       preds.append(output_text[i])
#       real.append(target[i])

  
  # return preds, real
# print(TestSet["Text"])

In [ ]:
preds = []
real = []
device = 'cpu'

def testModel(dataSet, bs):
  # model = t5_base.get_model()
  t5_base = T5_BASE_GENERATION
  transform = t5_base.transform()
  model = t5_base.get_model()
  sequence_generator = GenerationUtils(model)
  model.eval()
  sequence_generator = GenerationUtils(model)
  model.to(device)

  #turn the inputted dataframe into an iterable dataloader
  temp = DataLoader(dataSet, batch_size=bs)
  #temp.to(device)
  batch = next(iter(temp))
  #split the data into text and labels
  input_text = batch["Text"]
  target = batch["Class"]
  beam_size = 1

 # print("Pre val")
  #not 100% sure about excactly how this works, but it runs the model correctly in cell 7, uncomment cell 7 to see output.
  model_input = transform(input_text)
  model_output = sequence_generator.generate(model_input, eos_idx=1, num_beams=beam_size, max_length = 256)
  output_text = transform.decode(model_output.tolist())
  #print("Post val")

  for i in range(bs):
    # print(f"prediction: {output_text[i]}\n")
    # print(f"target: {target[i]}\n\n")
    #append results to list
    preds.append(output_text[i])
    real.append(target[i])
  #deleting local vars
  del batch
  del model_output
  del model_input
  del output_text
  del sequence_generator
  del temp
  gc.collect()
  #torch.cpu.empty_cache()


# preds, reals = predBatch(TestSet, 2)

k = 0
bs = 10

#loop throuhg whole test set
while(k < len(test_labels)):
  
  #prevent out of bounds
  if (bs > (len(test_labels) - k)):
    bs = len(test_labels) - k
  
  #grabbing relevant subset of data
  text_subset = test_text[k:k+bs]
  label_subset = test_labels[k:k+bs]
  
  #put into dataframe
  temp = pd.DataFrame({'Text': text_subset, 'Labels': label_subset})
  #print(temp)
  #adding neccassary prefix for sentiment analysis
  task = "sst2 sentence"
  for i in range(bs):
    temp["Text"][i] = task + ": " +  temp["Text"][i]

  #create custom class
  # torch_tensor = torch.tensor(temp['Text'].values)
  # torch_tensor.to(device)
  subset = CustomTextDataset(temp['Text'],temp['Labels'])
  #run the test

  testModel(subset, bs)

  del subset
  del temp
  gc.collect()
  k += bs
  print(k)


NameError: ignored

In [ ]:
print(len(preds))